In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from pathlib import Path
import re
import random
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Reading the BF data

In [3]:
bf_path = '/content/drive/MyDrive/ALL_CLEAN_DEIDEN_NAME AND ECMO DATA(Sheet1) (1) (version 2).csv'
cols = ['ID', 'age_days', 'weight', 'study_height', 'Diagnosis']
df = pd.read_csv(bf_path, usecols=cols)

for col in ['weight', 'study_height', 'age_days']:
    s = pd.to_numeric(df[col], errors='coerce')
    mx = s.max(skipna=True)
    if pd.notna(mx) and mx != 0:
        df[col] = s / mx
    else:
        df[col] = s
print(df.head())

   ID  age_days            Diagnosis    weight  study_height
0   1  0.018213       Cardiac Arrest  0.042214      0.344538
1   2  0.072852       Cardiac Arrest  0.070000      0.431373
2   3  0.793776               Sepsis  0.403571      0.896359
3   4  0.000287  Respiratory Failure  0.022786      0.268908
4   5  0.000287  Respiratory Failure  0.023571      0.285714


In [4]:
diag_clean = (
    df['Diagnosis']
      .astype('string')
      .str.strip()
      .str.replace(r'\s+', ' ', regex=True)
      .fillna('Unknown')
)

codes, uniques = pd.factorize(diag_clean, sort=True)
df['Diagnosis'] = codes.astype('int64')
diagnosis_mapping = {cat: int(i) for i, cat in enumerate(uniques)}
print("Diagnosis mapping (category -> code):", diagnosis_mapping)
# Peek
print(df.head())

Diagnosis mapping (category -> code): {'Cardiac Arrest': 0, 'Cardiogenic Shock': 1, 'Respiratory Failure': 2, 'Respiratory Failure, Cardiac Arrest': 3, 'Sepsis': 4, 'Septic shock': 5}
   ID  age_days  Diagnosis    weight  study_height
0   1  0.018213          0  0.042214      0.344538
1   2  0.072852          0  0.070000      0.431373
2   3  0.793776          4  0.403571      0.896359
3   4  0.000287          2  0.022786      0.268908
4   5  0.000287          2  0.023571      0.285714


In [5]:
df.shape

(73, 5)

### No normalization!

In [6]:
from pathlib import Path
import re, random, math, os
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# =========================
# Config
# =========================
SPLIT_DIR = r"/content/drive/MyDrive/CD/patient_data_clean_nozero_181920212223_1800"
POS_PATIENTS = {1, 2, 16, 19, 21, 22, 25, 37, 39, 43, 44, 47, 50, 56, 58, 62, 65, 66, 73, 78}

BATCH_SIZE       = 3
EPOCHS           = 100
LR               = 1e-4
SEED             = 1

# =========================
# Repro
# =========================
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
for g in tf.config.list_physical_devices('GPU'):
    try: tf.config.experimental.set_memory_growth(g, True)
    except Exception: pass

# =========================
# Helpers
# =========================
PATIENT_NUM_RX = re.compile(r'^ID(\d+)')  # e.g., "ID76-2_..." -> 76
def patient_num_from_path(pathlike):
    stem = Path(pathlike).stem
    m = PATIENT_NUM_RX.match(stem)
    return int(m.group(1)) if m else None

def label_for_file(p: Path) -> int:
    pnum = patient_num_from_path(p)
    return 1 if (pnum is not None and pnum in POS_PATIENTS) else 0

# =========================
# Load pre-existing 4-feature DataFrame: df (must be in memory)
# Must contain column 'ID' + 4 feature columns.
# =========================
feats_df = df.copy()  # uses your in-memory DataFrame
if "ID" not in feats_df.columns:
    raise RuntimeError("Your features DataFrame must contain column 'ID'.")

FEAT_COLS = [c for c in feats_df.columns if c != "ID"]
# if len(FEAT_COLS) != 4:
#     raise RuntimeError(f"Expected exactly 4 feature columns, found {len(FEAT_COLS)}: {FEAT_COLS}")

feats_df["ID"] = pd.to_numeric(feats_df["ID"], errors="coerce").astype("Int64")
feats_df = feats_df.dropna(subset=["ID"] + FEAT_COLS).copy()
feats_df["ID"] = feats_df["ID"].astype(int)

ID_TO_FEAT = {
    int(row["ID"]): row[FEAT_COLS].astype("float32").to_numpy()
    for _, row in feats_df.iterrows()
}
FEAT_DIM = len(FEAT_COLS)
print('FEAT_DIM =', FEAT_DIM)

# =========================
# List EEG files ONLY to define splits by patient ID (no EEG is loaded)
# =========================
split_dir = Path(SPLIT_DIR)
all_csvs = sorted(split_dir.glob("*.csv"))
if not all_csvs:
    raise FileNotFoundError(f"No CSV found in {SPLIT_DIR}")

id_to_files = {}
for f in all_csvs:
    pid = patient_num_from_path(f)
    if pid is None:
        continue
    id_to_files.setdefault(pid, []).append(f)

all_ids = sorted(id_to_files.keys())

valid_ids = [pid for pid in all_ids if pid in ID_TO_FEAT]
if not valid_ids:
    raise RuntimeError("No overlapping patient IDs between files and the 4-feature table.")
if len(valid_ids) < len(all_ids):
    print(f"Dropping {len(all_ids)-len(valid_ids)} patient IDs without 4-feature rows.")

labels_all = np.array([1 if pid in POS_PATIENTS else 0 for pid in valid_ids], dtype=int)

print("Total valid IDs:", len(valid_ids),
      "| Pos IDs:", labels_all.sum(),
      "| Neg IDs:", (1 - labels_all).sum())

# =========================
# Data Sequence (tab-only)
# Each file becomes one sample with that patient's features.
# =========================
class TabSequence(keras.utils.Sequence):
    def __init__(self, files, batch_size=BATCH_SIZE, shuffle=True):
        super().__init__()
        self.files = [f for f in files if patient_num_from_path(f) in ID_TO_FEAT]
        self.batch_size = int(batch_size)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.files) / self.batch_size)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.files))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        idxs = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_files = [self.files[i] for i in idxs]
        B = len(batch_files)

        X_tab = np.empty((B, FEAT_DIM), dtype=np.float32)
        y     = np.empty((B,), dtype=np.int32)

        for i, f in enumerate(batch_files):
            pid = patient_num_from_path(f)
            X_tab[i] = ID_TO_FEAT[pid]
            y[i] = label_for_file(f)

        return {"tab_input": X_tab}, y

# =========================
# Tab-only Model: 4 -> 8 -> 8 -> 1
# =========================
def build_model(tab_dim=FEAT_DIM, lr=LR, dropout=0.2):
    tab_in = keras.Input(shape=(tab_dim,), name="tab_input")
    t = layers.Dense(8, activation="relu")(tab_in)
    t = layers.Dense(8, activation="relu")(t)
    t = layers.Dropout(dropout)(t)
    out = layers.Dense(1, activation="sigmoid")(t)

    model = keras.Model(inputs=tab_in, outputs=out)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[keras.metrics.BinaryAccuracy(name="acc"),
                 keras.metrics.AUC(name="auc")],
    )
    return model

# =========================
# Utilities
# =========================
def safe_roc_auc(y_true, probs):
    try:
        return roc_auc_score(y_true, probs)
    except ValueError:
        return float('nan')

def plot_roc(y_true, probs, title, out_png):
    try:
        fpr, tpr, _ = roc_curve(y_true, probs)
        plt.figure()
        auc = safe_roc_auc(y_true, probs)
        plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
        plt.plot([0,1],[0,1], linestyle="--", linewidth=1)
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(title)
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_png, dpi=200)
        plt.close()
    except Exception as e:
        print(f"(Warning) ROC plot failed ({title}): {e}")

# =========================
# 10-fold Cross-Validation by patient ID (stratified)
# =========================
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

fold_val_aucs, fold_val_accs = [], []
fold_test_aucs, fold_test_accs = [], []
fold_sizes = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(valid_ids, labels_all), start=1):
    ids_train_full = [valid_ids[i] for i in train_index]
    ids_test       = [valid_ids[i] for i in test_index]

    # small validation split from training IDs (stratified, by ID)
    train_labels_full = np.array([1 if pid in POS_PATIENTS else 0 for pid in ids_train_full], dtype=int)
    ids_tr, ids_val = train_test_split(
        ids_train_full, test_size=0.10, random_state=SEED,
        stratify=train_labels_full
    )

    # Build file lists for this fold
    train_files = [f for pid in ids_tr  for f in id_to_files[pid]]
    val_files   = [f for pid in ids_val for f in id_to_files[pid]]
    test_files  = [f for pid in ids_test for f in id_to_files[pid]]

    print(f"\n--- Fold {fold_idx}/10 ---")
    def split_summary(name, ids, files):
        ys = np.array([label_for_file(f) for f in files], dtype=int)
        print(f"{name:>6} | ids: {len(ids):4d} | files: {len(files):4d} | pos: {(ys==1).sum():4d} | neg: {(ys==0).sum():4d}")
    split_summary("train", ids_tr,  train_files)
    split_summary("val",   ids_val, val_files)
    split_summary("test",  ids_test, test_files)

    fold_sizes.append((len(train_files), len(val_files), len(test_files)))

    # Generators
    train_gen = TabSequence(train_files, batch_size=BATCH_SIZE, shuffle=True)
    val_gen   = TabSequence(val_files,   batch_size=BATCH_SIZE, shuffle=False)

    # Model + training
    model = build_model()
    best_path = f"best_tab_only_fold{fold_idx}.h5"
    ckpt = keras.callbacks.ModelCheckpoint(
        best_path, monitor="val_loss", mode="min", save_best_only=True, verbose=1
    )

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=[ckpt],
        verbose=1,
    )

    # Load best
    best_model = keras.models.load_model(best_path)

    # ====== VALIDATION METRICS (like previous script) ======
    val_probs = best_model.predict(val_gen, verbose=0).ravel().astype(float)
    val_ytrue = np.array([label_for_file(f) for f in val_gen.files], dtype=int)
    val_auc = safe_roc_auc(val_ytrue, val_probs)
    val_acc = accuracy_score(val_ytrue, (val_probs >= 0.5).astype(int))
    fold_val_aucs.append(val_auc)
    fold_val_accs.append(val_acc)
    print(f"Fold {fold_idx} | VAL  | AUC={val_auc:.4f} | ACC={val_acc:.4f} | n={len(val_ytrue)}")
    plot_roc(val_ytrue, val_probs,
             title=f"ROC — VAL Fold {fold_idx:02d} (n={len(val_ytrue)})",
             out_png=f"roc_val_fold_{fold_idx:02d}.png")

    # ====== TEST METRICS (keeps your original test evaluation) ======
    test_files2 = [f for f in test_files if patient_num_from_path(f) in ID_TO_FEAT]
    X_tab_test = np.empty((len(test_files2), FEAT_DIM), dtype=np.float32)
    for i, f in enumerate(test_files2):
        X_tab_test[i] = ID_TO_FEAT[patient_num_from_path(f)]
    y_true = np.array([label_for_file(f) for f in test_files2], dtype=int)

    probs = best_model.predict({"tab_input": X_tab_test}, verbose=0).ravel().astype(float)
    test_auc = safe_roc_auc(y_true, probs)
    test_acc = accuracy_score(y_true, (probs >= 0.5).astype(int))
    fold_test_aucs.append(test_auc)
    fold_test_accs.append(test_acc)
    print(f"Fold {fold_idx} | TEST | AUC={test_auc:.4f} | ACC={test_acc:.4f} | n={len(y_true)}")
    plot_roc(y_true, probs,
             title=f"ROC — TEST Fold {fold_idx:02d} (n={len(y_true)})",
             out_png=f"roc_test_fold_{fold_idx:02d}.png")

# =========================
# Results across folds
# =========================
def mean_std(arr):
    arr = np.asarray(arr, dtype=float)
    return np.nanmean(arr), np.nanstd(arr)

mAUC_val, sAUC_val = mean_std(fold_val_aucs)
mACC_val, sACC_val = mean_std(fold_val_accs)
mAUC_tst, sAUC_tst = mean_std(fold_test_aucs)
mACC_tst, sACC_tst = mean_std(fold_test_accs)

print("\nPer-fold VAL  AUCs:", [None if np.isnan(x) else round(x,4) for x in fold_val_aucs])
print("Per-fold VAL  ACCs:", [round(x,4) for x in fold_val_accs])
print("Per-fold TEST AUCs:", [None if np.isnan(x) else round(x,4) for x in fold_test_aucs])
print("Per-fold TEST ACCs:", [round(x,4) for x in fold_test_accs])

print(f"\nVAL  AUC: {mAUC_val:.4f} ± {sAUC_val:.4f} | ACC: {mACC_val:.4f} ± {sACC_val:.4f}")
print(f"TEST AUC: {mAUC_tst:.4f} ± {sAUC_tst:.4f} | ACC: {mACC_tst:.4f} ± {sACC_tst:.4f}")

# Save fold-wise metrics
rows = []
for i, (tr_n, va_n, te_n) in enumerate(fold_sizes, start=1):
    rows.append({
        "fold": i,
        "train_files": tr_n,
        "val_files": va_n,
        "test_files": te_n,
        "val_auc": fold_val_aucs[i-1],
        "val_acc": fold_val_accs[i-1],
        "test_auc": fold_test_aucs[i-1],
        "test_acc": fold_test_accs[i-1],
    })
metrics_df = pd.DataFrame(rows)
metrics_df.to_csv("cv_tabonly_fold_metrics.csv", index=False)
print("\nSaved metrics to cv_tabonly_fold_metrics.csv and ROC plots to roc_val_fold_XX.png / roc_test_fold_XX.png")


FEAT_DIM = 4
Total valid IDs: 44 | Pos IDs: 14 | Neg IDs: 30

--- Fold 1/10 ---
 train | ids:   35 | files:  904 | pos:  318 | neg:  586
   val | ids:    4 | files:   42 | pos:    1 | neg:   41
  test | ids:    5 | files:  254 | pos:   86 | neg:  168
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: tab_input
Received: inputs=['Tensor(shape=(None, 4))']
  warnings.warn(msg)


288/302 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.6052 - auc: 0.5564 - loss: 0.6613
Epoch 1: val_loss improved from inf to 0.47449, saving model to best_tab_only_fold1.h5


302/302 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - acc: 0.6060 - auc: 0.5567 - loss: 0.6611 - val_acc: 0.9762 - val_auc: 1.0000 - val_loss: 0.4745
Epoch 2/100
288/302 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.6104 - auc: 0.5556 - loss: 0.6544
Epoch 2: val_loss improved from 0.47449 to 0.43209, saving model to best_tab_only_fold1.h5


302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.6114 - auc: 0.5577 - loss: 0.6538 - val_acc: 0.9762 - val_auc: 1.0000 - val_loss: 0.4321
Epoch 3/100
288/302 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.6393 - auc: 0.6164 - loss: 0.6307
Epoch 3: val_loss improved from 0.43209 to 0.39838, saving model to best_tab_only_fold1.h5


...

Fold 9 | VAL  | AUC=0.9077 | ACC=0.6280 | n=207
Fold 9 | TEST | AUC=1.0000 | ACC=1.0000 | n=149

--- Fold 10/10 ---
 train | ids:   36 | files:  874 | pos:  319 | neg:  555
   val | ids:    4 | files:  145 | pos:   77 | neg:   68
  test | ids:    4 | files:  181 | pos:    9 | neg:  172
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: tab_input
Received: inputs=['Tensor(shape=(None, 4))']
  warnings.warn(msg)


274/292 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.6733 - auc: 0.6013 - loss: 0.6686
Epoch 1: val_loss improved from inf to 0.79471, saving model to best_tab_only_fold10.h5


292/292 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - acc: 0.6721 - auc: 0.6004 - loss: 0.6689 - val_acc: 0.2000 - val_auc: 0.2059 - val_loss: 0.7947
Epoch 2/100
287/292 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.6896 - auc: 0.6194 - loss: 0.6502
Epoch 2: val_loss did not improve from 0.79471
292/292 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.6890 - auc: 0.6192 - loss: 0.6503 - val_acc: 0.4690 - val_auc: 0.4118 - val_loss: 0.7955
Epoch 3/100
287/292 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.6779 - auc: 0.6606 - loss: 0.6357
Epoch 3: val_loss did not improve from 0.79471
292/292 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.6774 - auc: 0.6603 - loss: 0.6359 - val_acc: 0.4690 - val_auc: 0.4118 - val_loss: 0.7990
Epoch 4/100
287/292 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.6405 - auc: 0.5746 - loss: 0.6623
Epoch 4: val_loss did not improve from 0.79471
292/292 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.6408 - auc: 0.5756 - loss: 0.6620 - val_acc: 0.4690 - val_auc: 0.4118 - val_loss: 0.8004
Epoch 5/100
28

Fold 10 | VAL  | AUC=0.0000 | ACC=0.2000 | n=145
Fold 10 | TEST | AUC=1.0000 | ACC=1.0000 | n=181

Per-fold VAL  AUCs: [np.float64(1.0), np.float64(0.4118), np.float64(1.0), np.float64(0.0), np.float64(0.4118), np.float64(1.0), np.float64(0.049), np.float64(0.8957), np.float64(0.9077), np.float64(0.0)]
Per-fold VAL  ACCs: [0.9762, 0.6415, 0.9811, 0.7901, 0.469, 0.6709, 0.65, 0.599, 0.628, 0.2]
Per-fold TEST AUCs: [np.float64(0.0554), np.float64(0.91), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.6316), np.float64(0.155), np.float64(1.0), np.float64(1.0), np.float64(1.0)]
Per-fold TEST ACCs: [0.6614, 0.5, 0.5207, 0.9804, 0.678, 0.9744, 0.2246, 1.0, 1.0, 1.0]

VAL  AUC: 0.5676 ± 0.4179 | ACC: 0.6606 ± 0.2176
TEST AUC: 0.7752 ± 0.3528 | ACC: 0.7539 ± 0.2636

Saved metrics to cv_tabonly_fold_metrics.csv and ROC plots to roc_val_fold_XX.png / roc_test_fold_XX.png


In [8]:
rows

[{'fold': 1,
  'train_files': 904,
  'val_files': 42,
  'test_files': 254,
  'val_auc': np.float64(1.0),
  'val_acc': 0.9761904761904762,
  'test_auc': np.float64(0.05537098560354374),
  'test_acc': 0.6614173228346457},
 {'fold': 2,
  'train_files': 1014,
  'val_files': 106,
  'test_files': 80,
  'val_auc': np.float64(0.4117647058823529),
  'val_acc': 0.6415094339622641,
  'test_auc': np.float64(0.91),
  'test_acc': 0.5},
 {'fold': 3,
  'train_files': 978,
  'val_files': 53,
  'test_files': 169,
  'val_auc': np.float64(1.0),
  'val_acc': 0.9811320754716981,
  'test_auc': np.float64(1.0),
  'test_acc': 0.5207100591715976},
 {'fold': 4,
  'train_files': 968,
  'val_files': 181,
  'test_files': 51,
  'val_auc': np.float64(0.0),
  'val_acc': 0.7900552486187845,
  'test_auc': np.float64(1.0),
  'test_acc': 0.9803921568627451},
 {'fold': 5,
  'train_files': 996,
  'val_files': 145,
  'test_files': 59,
  'val_auc': np.float64(0.4117647058823529),
  'val_acc': 0.4689655172413793,
  'test_auc':

In [10]:

test_auc = np.array([r["test_auc"] for r in rows], dtype=float)
test_acc = np.array([r["test_acc"] for r in rows], dtype=float)

print(f"Mean TEST AUC: {np.nanmean(test_auc):.4f}")
print(f"Mean TEST ACC: {np.nanmean(test_acc):.4f}")


Mean TEST AUC: 0.7752
Mean TEST ACC: 0.7539
